<a href="https://colab.research.google.com/github/dave20874/RapidReact-Kicker/blob/main/Kicker-Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
import math

This notebook develops some ideas for a pinball-style kicker to launch CARGO (oversized tennis balls) as part of 2022 FIRST Robotics Competition game, Rapid React.

The analysis relies on the concept of conservation of energy.  A motor will push back plunger (performing work), compressing a spring (storing potential energy).  When the plunger is released, the spring propels the plunger forward (converting to kinetic energy).  At the end of it's stroke, the plunger strikes the ball, transferring some kinetic energy to it.  The ball then flies up, it's energy transitioning smoothly to gravitational potential energy as it rises.

Let's begin by defining some values we'll be using:


In [206]:
# Gravitational acceleration, meters per second
# (For FRC matches held on Earth)
g = 9.8

# mass of the CARGO ball, kg
m_ball = 0.27

# radius of CARGO ball, meters
ball_radius = 0.12

# In a collision, the ball only retains about 80% of it's kinetic energy.
ball_loss = 0.80

# Initial height of ball before shot, meters
h_shooter = 1.0

# Goal height, meters.  (8 feet 8 inches)
h_goal = 2.64

# Upper goal radius, meters
goal_radius = 0.61

prototype = "2x4 proto 2"

if prototype == "2x4 proto 1":
  # mass of the plunger, kg
  m_plunger = 1.31  # 2x4 plunger
  # m_plunger = 0.400

  # pull distance of the plunger, meters
  d_plunger = 0.15

  # The stretch spring stretches 12.8 cm with 20 lbs (9.09 kg) on it.
  # (696 N/m)
  k_stretch = 9.09*g/0.128

  # spring constant, Newtons per meter
  # We will use 6 springs and measured 1.5 kg of force at 1cm (0.01m) compression
  k_one_spring = k_stretch
  num_springs = 1
elif prototype == "2x4 proto 2":
  # mass of the plunger, kg
  # m_plunger = 1.31  # 2x4 plunger
  m_plunger = 0.400

  # pull distance of the plunger, meters
  d_plunger = 0.13

  # The stretch spring stretches 12.8 cm with 20 lbs (9.09 kg) on it.
  # (696 N/m)
  k_stretch = 9.09*g/0.128

  # spring constant, Newtons per meter
  # We will use 6 springs and measured 1.5 kg of force at 1cm (0.01m) compression
  k_one_spring = k_stretch
  num_springs = 2
elif prototype == "2x4 proto 3":
  # mass of the plunger, kg
  # m_plunger = 1.31  # 2x4 plunger
  m_plunger = 0.400

  # pull distance of the plunger, meters
  d_plunger = 0.13

  # The stretch spring stretches 12.8 cm with 20 lbs (9.09 kg) on it.
  # (696 N/m)
  k_stretch = 9.09*g/0.128

  # spring constant, Newtons per meter
  # We will use 6 springs and measured 1.5 kg of force at 1cm (0.01m) compression
  k_one_spring = k_stretch
  num_springs = 3
elif prototype == "compression 6":
  # Design goal
  # mass of the plunger, kg
  m_plunger = 0.400

  # pull distance of the plunger, meters
  d_plunger = 0.06

  # The stretch spring stretches 1 cm with 1.5 kg on it.
  # (1470 N/m)
  k_stretch = 1.5*g/0.01

  # spring constant, Newtons per meter
  k_one_spring = k_stretch
  num_springs = 6
else:
  raise Exception("No kicker design specified")

k_spring = num_springs * k_one_spring



Now we can define some useful functions.  These functions compute the amount of energy in various parts of our system and convert energy back into values like distances and speeds.

In [207]:
# Spring stored energy
# For a linear spring with spring constant, k, compressed by distance d.
def spring_energy(k, d):
  energy = 0.5 * k * d * d
  return energy

# Compute spring stretch amount for a spring with constant, k, and energy e
# since spring energy, e = 1/2 * k * d^2, we have d = sqrt(2*e/k)
def spring_stretch(k, e):
  d = math.sqrt(2.0*e/k)
  return d

# Get kinetic energy of a mass, m, moving at velocity, v
# e = 1/2 * m * v^2
def vel_to_ke(m, v):
  e = 0.5 * m * v * v
  return e

# Get velocity of a mass, m, with kinetic energy e
# since kinetic energy, e = 1/2*m*v^2, we have v = sqrt(2*e/m)
def ke_to_vel(e, m):
  v = math.sqrt(2.0 * e / m)
  return v

# Get potential energy of a mass, m, at height, h
def h_to_pe(m, h):
  e = m * g * h
  return e

# Get height of a mass, m, with potential energy e
def pe_to_h(e, m):
  h = e / (m * g)
  return h
  

Now we can ask some questions.

How much energy does the spring store, when it's compressed?  This is the amount of energy we're starting with.

In [208]:
# Starting energy
e = spring_energy(k_spring, d_plunger)
print(f"Starting energy: {e:4.3f} (Design: {prototype})")


Starting energy: 11.762 (Design: 2x4 proto 2)


And, quick question, how high would a CARGO go if all this energy were transferred to it?

In [209]:
# Height of a ball with this much energy added to it.
h = pe_to_h(e, m_ball)
print(f"How high could we shoot, above launcher? {h:4.3f} meters")
print(f"That is {h+h_shooter:4.3f} meters above floor")
print(f"And {h+h_shooter-h_goal:4.3f} meters above the high goal.")


How high could we shoot, above launcher? 4.445 meters
That is 5.445 meters above floor
And 2.805 meters above the high goal.


Using energy, let's compute the height of a launched ball directly from the compression of a spring.  Nice!  But we need to take a deeper look at the whole launch process.  What if not all that energy goes into the ball?  And what if we are launching the ball at an angle instead of straight up?

First, lets look at the collision of the plunger with the ball.  Does all the energy go to the ball?  No!  How much the ball receives depends on the mass of the plunger and the mass of the ball.

If there is no friction in the plunger mechanism, we can assume all the spring energy is transferred to the plunger just before it strikes the ball.  So, knowing the spring's energy and the plunger's mass, we can compute the speed of the plunger.

In [210]:
v_plunger = ke_to_vel(e, m_plunger)
print(f"Plunger is moving {v_plunger:4.3f} meters per second")

Plunger is moving 7.669 meters per second


Then we use an equation for computing the effect of a collision between two masses in one dimension.  

In this formula, m1 and m2 are the masses of two objects.  Before the collision they have velocities u1 and u2.  After, they have velocities v1 and v2.

We will evaluate the kinetic energy of the ball and plunger at this point to see how efficient our collision was.

In [211]:
# Compute post-collision velocities, v1 and v2, for objects with mass m1, m2 colliding with velocities u1, u2
def collide(m1, u1, m2, u2):
  v1 = (u1*(m1-m2)+2*m2*u2) / (m1+m2)
  v2 = (u2*(m2-m1)+2*m1*u1) / (m1+m2)
  return (v1, v2)

(v1, v2) = collide(m_plunger, v_plunger, m_ball, 0.0)
print(f"After an elastic collision, the ball would have velocity {v2:4.3f} meters per second")
print(f"The plunger would have velocity {v1:4.3f} meters per second.")
e_ball = vel_to_ke(m_ball, v2)
e_plunger = vel_to_ke(m_plunger, v1)
print(f"The ball has {e_ball:4.3f} Joules after collision.")
print(f"That is {100*e_ball/e:4.3f}% of the energy we started with.")

After an elastic collision, the ball would have velocity 9.157 meters per second
The plunger would have velocity 1.488 meters per second.
The ball has 11.319 Joules after collision.
That is 96.235% of the energy we started with.


TODO: Add analysis for optimal plunger mass.

We know the ball dissipates energy in collisions, though.  So we should reduce it's energy and re-evaluate how high it might go

In [212]:
e_ball = e_ball * ball_loss
print(f"Ball's adjusted energy: {e_ball:4.3f} Joules")
v_ball = ke_to_vel(e_ball, m_ball)
print(f"Ball's adjusted speed: {v_ball:4.3f} meters per second")
h_max = h_shooter + pe_to_h(e_ball, m_ball)
print(f"It can reach a max height of {h_max:4.3f} meters above the floor.")
print(f"That is {h_max-h_goal:4.3f} meters above the goal.")

Ball's adjusted energy: 9.055 Joules
Ball's adjusted speed: 8.190 meters per second
It can reach a max height of 4.422 meters above the floor.
That is 1.782 meters above the goal.


What sort of trajectories are possible with this much ball velocity?  We can answer this by looking first at the ball's velocity when it is at the opening of the high goal.  We can evaluate the potential energy at the goal high, subtract that from the total energy and the remaining part must be kinetic energy.  From there we get the velocity.

In [213]:
pe_goal = h_to_pe(m_ball, h_goal)
pe_shooter = h_to_pe(m_ball, h_shooter)
print(f"Potential energy at goal: {pe_goal} Joules")
ke_at_goal = pe_shooter + e_ball - pe_goal
v_at_goal = ke_to_vel(ke_at_goal, m_ball)
print(f"Velocity at goal entrance: {v_at_goal:4.3f} meters per second")


Potential energy at goal: 6.985440000000001 Joules
Velocity at goal entrance: 5.910 meters per second


Next, we can evaluate trajectories with that velocity, entering the goal at angles from vertical (90 degrees) to horizontal (0 degrees).  For each landing angle, we'll compute the max height above the goal, the horizontal distance from the goal when ascending, the launch distance from center of the field and the launch angle from the robot.  Some of these trajectories will be invalid because they don't clear the edge of the goal (max height less than ball radius horizontal ascent distance less than goal radius)

In [214]:
# Given the goal velocity and angle, compute trajectory data:
# returns (clearance_z, clearance_x, launch_d, launch_angle, is_valid)
def eval_trajectory(final_v, final_theta, launch_h):
  v_z = math.sin(final_theta)*final_v                # vertical speed at goal
  v_x = math.cos(final_theta)*final_v                # horizontal speed at goal
  t_apex_goal = v_z / g                         # time from apex to goal
  z_above_goal = 0.5 * g * t_apex_goal * t_apex_goal # height of apex above goal
  clearance_z = z_above_goal - ball_radius
  x_at_goal = 2.0 * t_apex_goal * v_x           # hor movement from h_goal to apex and back
  clearance_x = x_at_goal - goal_radius - ball_radius
  t_shot_apex = math.sqrt(2.0 * (h_goal+z_above_goal-h_shooter) / g)
  v_z_shooter = t_shot_apex * g
  launch_d = v_x * (t_shot_apex+t_apex_goal)
  launch_angle = math.atan(v_z_shooter / v_x)

  is_valid = True
  if clearance_z <= 0:
    is_valid = False
  if clearance_x <= 0:
    is_valid = False

  return (clearance_z, clearance_x, launch_d, launch_angle, is_valid)

print(f"{prototype}")
print(f"k_spring: {k_one_spring} N/m")
print(f"num springs: {num_springs}")
print(f"draw dist: {d_plunger} m")
print(f"plunger mass: {m_plunger} kg")
print("   entry z-clear x-clear    dist   angle   valid")
for degrees in range(90, 28, -2):
  theta = math.radians(degrees)
  cz, cx, dist, angle, valid = eval_trajectory(v_at_goal, theta, h_shooter)
  print(f"{math.degrees(theta):8.3f}{cz:8.3f}{cx:8.3f}{dist:8.3f}{math.degrees(angle):8.3f}{valid:8}")

2x4 proto 2
k_spring: 695.953125 N/m
num springs: 2
draw dist: 0.13 m
plunger mass: 0.4 kg
   entry z-clear x-clear    dist   angle   valid
  90.000   1.662  -0.730   0.000  90.000       0
  88.000   1.660  -0.481   0.297  88.557       0
  86.000   1.653  -0.234   0.592  87.115       0
  84.000   1.643   0.011   0.885  85.674       1
  82.000   1.628   0.252   1.175  84.236       1
  80.000   1.608   0.489   1.460  82.801       1
  78.000   1.585   0.720   1.740  81.371       1
  76.000   1.558   0.943   2.013  79.946       1
  74.000   1.527   1.159   2.279  78.527       1
  72.000   1.492   1.365   2.535  77.115       1
  70.000   1.454   1.561   2.783  75.711       1
  68.000   1.412   1.746   3.019  74.316       1
  66.000   1.367   1.919   3.245  72.931       1
  64.000   1.320   2.079   3.458  71.558       1
  62.000   1.269   2.225   3.659  70.197       1
  60.000   1.217   2.357   3.847  68.849       1
  58.000   1.162   2.474   4.020  67.517       1
  56.000   1.105   2.575   